<a href="https://colab.research.google.com/github/almuller3/MLDL/blob/main/Vae_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# VAE Model
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2_mean = nn.Linear(512, latent_dim)
        self.fc2_logvar = nn.Linear(512, latent_dim)
        # Decoder
        self.fc3 = nn.Linear(latent_dim, 512)
        self.fc4 = nn.Linear(512, input_dim)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc2_mean(h1), self.fc2_logvar(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Loss Function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='mean')
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Parameters
input_dim = 784
latent_dim = 20
epochs = 10
log_interval = 100

# Data Transformations
transform = transforms.Compose([transforms.ToTensor()])

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)

# Model
model = VAE(input_dim, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device).float()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item() * len(data)
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')

# Training
for epoch in range(1, epochs + 1):
    train(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.707682
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.267047
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.272443
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.255493
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.263862
====> Epoch: 1 Average loss: 0.2748
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.264731
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.268995
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.272343
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.262417
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.263998
====> Epoch: 2 Average loss: 0.2646
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.259509
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.254085
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.257850
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.262163
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.262418
====> Epoch: 3 Average loss: 0.2643
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.266838
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.267125
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.261996
Train Epoch:

Varb

# New Section